# 2. 2D XRF deconvolution of pre-processed hdf files
## Summary
This notebook takes the pre-processed .hdf files produced with the `1_reduced_reshaped_xrf_hdf` notebook and deconvolutes to element channel images using the PyMCA package.

In [1]:
import pathlib
import shutil
import numpy as np
import pandas as pd
import h5py

from PyMca5.PyMcaPhysics.xrf.XRFBatchFitOutput import OutputBuffer
from PyMca5.PyMcaPhysics.xrf.FastXRFLinearFit import FastXRFLinearFit

In [2]:
# Set data directory to work from 
base_dir = "C:/Users/MerrickS/OneDrive/Work/2_UZH/Papers/1_MEZ_XRF"
base_dir = pathlib.Path(base_dir)

# Specify the input directory where hdf files to process are located
hdf_dir = base_dir / 'data' / 'processed' / 'xrf' / '1_reduced_reshaped_hdfs'

# Specify the config directory where config files for deconvolution are located
cfg_dir = base_dir / 'data' / 'raw' / 'xrf' / 'config'

# Make output directory for reshaped hdf files if it does not exist
out_dir = base_dir / 'data' / 'processed' / 'xrf' / '2_deconvoluted_hdfs'
out_dir.mkdir(parents=True, exist_ok=True)
print('\n Deconvoluted hdf files will be output to: \n\t', base_dir, out_dir) 

# Gather filepaths for preprocessed hdfs and config files for XRF fitting
hdf_filepaths = list(hdf_dir.glob('*.h5'))

# Read in scan and scanset metadata
df_hdf_config = pd.read_csv(hdf_dir /'preprocessed_hdf_config_files.csv')


 Deconvoluted hdf files will be output to: 
	 C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs


In [4]:
# Make hdf XRF fit config dictionary 
hdf_list = df_hdf_config['hdf_file'].tolist()
config_list = df_hdf_config['config_file'].tolist()
hdf_config_dict = dict(zip(hdf_list, config_list))

In [8]:
def hdf_dataset(hdf_filepath, dataset): 
    with h5py.File(hdf_filepath, 'r') as hdf:
        node = f"{dataset}"
        try:
            hdf[node]
            dset = hdf[dataset]
        except KeyError:
            dset = []
            print(f'Could not find hdf dataset for {hdf_filepath}')

        dset = dset[:]
    return dset

def xrf_deconvolute(hdf_file, detector, refit_state=False, weight=0):
    # Check hdf file exists
    input_hdf_fpath = hdf_dir / f'{hdf_file}.h5'
    if input_hdf_fpath.exists() is False:
        print(f'{input_hdf_fpath} does not exist')
        
    # Check detector node exists
    node_exists = False
    with h5py.File(input_hdf_fpath, 'r+') as hdf:  
        det_entries = len(hdf[detector][:])
        print(det_entries)
        if det_entries > 100:
            node_exists = True
        else:
            print(detector, 'empty, no fit performed')

         #   node_exists = True
          #  print('node exists')
    
    # Check cfg file exists
    config_filepath = cfg_dir / hdf_config_dict[hdf_file]
    if config_filepath.exists() is False:
        print(f'{config_filepath} does not exist')
        
    # Make output subdir if does not exist
    out_sub_dir = out_dir / 'per_detector_deconvolutions' / hdf_file / detector
    out_sub_dir.mkdir(parents=True, exist_ok=True)
        
    # Get dataset to deconvolute and reshape to 3D array (2D array of spectra) for PyMCA
    spectra_stack = hdf_dataset(input_hdf_fpath, detector)
    z_dim = len(list(set(hdf_dataset(input_hdf_fpath, 'hrz'))))
    y_dim = int(len(list(hdf_dataset(input_hdf_fpath, 'hry')))/z_dim)
    spectra_stack_sorted = spectra_stack.reshape(z_dim, y_dim, spectra_stack.shape[-1])  
    
    # Pymca fit
    if node_exists == True:
        pymca_object = FastXRFLinearFit()    
        
        print('spectra config file:', config_filepath.name)

        FastXRFLinearFit.setFitConfigurationFile(pymca_object, str(config_filepath))

        test_out = OutputBuffer(outputDir=out_sub_dir,
                                edf=0,
                                tif=True,
                                overwrite=True,
                                outputRoot=hdf_file, 
                                fileEntry=hdf_file
                                )
               
        FastXRFLinearFit.fitMultipleSpectra(pymca_object, 
                                            #x=spectra_stack[1],
                                            y=spectra_stack_sorted, 
                                            weight=weight, 
                                            refit=refit_state, 
                                            concentrations=False,
                                            outbuffer=test_out)
        
        # Carry over the fpico normalisation mask
        output_hdf_fpath = list(out_sub_dir.glob('*h5'))[0]
        
        with h5py.File(input_hdf_fpath, 'r') as hdf:
            if 'fpico_mask' in hdf:
                fpico_mask = hdf['fpico_mask'][:]
            else:
                fpico_mask = False
                
        print('output hdf file:', output_hdf_fpath)

        if isinstance(fpico_mask, np.ndarray):
            with h5py.File(output_hdf_fpath, 'r+') as hdf:
                  hdf.create_dataset(name = 'fpico_mask', data = fpico_mask)

    else:
        pass


Perform deconvolutions on both detectors (where present).

In [9]:
for hdf_file in hdf_list:
    print('\n', hdf_file)
    xrf_deconvolute(hdf_file, 'falconx_det0', refit_state=True)
    xrf_deconvolute(hdf_file, 'fluodet_det0', refit_state=True)


 appendix_a1_overview_solid_0001
219200


ValueError: cannot reshape array of size 897843200 into shape (176,1246,4096)

 Incorporate step sizes for final channel image hdf files. 

In [6]:
# Incorporate key csv attributes per scan to deconvoluted hdfs for subsequent analysis
hdf_img_fpaths = [i for i in out_dir.glob('*/*/*/*.h5')]

for hdf_img_fpath in hdf_img_fpaths:
    step = df_hdf_config.loc[df_hdf_config['hdf_file'] == hdf_img_fpath.stem, 'step_um'].iloc[0]
    det_type = df_hdf_config.loc[df_hdf_config['hdf_file'] == hdf_img_fpath.stem, 'detector'].iloc[0]
    dual_det = df_hdf_config.loc[df_hdf_config['hdf_file'] == hdf_img_fpath.stem, 'dual_detector'].iloc[0]
    
    if dual_det == 1:
        dual_det = 'yes'
    else:
        dual_det = 'no'
   
    with h5py.File(hdf_img_fpath, 'r+') as hdf:
        if 'pixel_um' in hdf:
            pass 
        else:
            hdf.create_dataset(name = 'detector', data = step)
        if 'detector' in hdf:
            pass 
        else:
            hdf.create_dataset(name = 'detector', data = det_type)
        if 'dual_detector' in hdf:
            pass 
        else:
            hdf.create_dataset(name = 'dual_detector', data = dual_det)

print('Key acquisition metadata incorporated to .hdf files')

Key acquisition metadata incorporated to .hdf files


Collate hdfs to summary hdfs per sample. For scans with dual detectors, element plots will be aggregated, while for single detector scans these will simply be copied over. 

In [7]:
detector_paths = ['falconx_det0', 'fluodet_det0']
out_sum_dir = out_dir / 'summary_hdfs'
out_sum_dir.mkdir(parents=True, exist_ok=True)

# Copy the falconX detector to summary scan output
for hdf_file in df_hdf_config['hdf_file']:
    hdf_dir = out_dir / 'per_detector_deconvolutions' / hdf_file 
    hdf_fpath = list(hdf_dir.glob('*/*.h5'))
    
    shutil.copyfile(hdf_fpath[0], out_sum_dir / hdf_fpath[0].name)
            
# For dual detector scans, modify the summary scan datasets by aggregating second detector data for plots
for hdf_file in df_hdf_config.loc[df_hdf_config['dual_detector'] == 1, 'hdf_file']:
    hdf_dir = out_dir / 'per_detector_deconvolutions' / hdf_file 
    hdf_fpath = list(hdf_dir.glob('*/*.h5'))
    
    hdf_base = h5py.File(hdf_fpath[0], 'r+')
    plots = list(hdf_base[f'{hdf_file}/plotselect'].keys())
    
    for plot in plots:
        dset = hdf_base[f'{hdf_file}/plotselect/{plot}'][:]
        
        with h5py.File(hdf_fpath[1], 'r+') as hdf_add:
            dset_add = hdf_add[f'{hdf_file}/plotselect/{plot}'][:]
            
        dset_mean = np.mean([dset, dset_add], axis=0)
        
        hdf_mod_path = out_sum_dir / hdf_fpath[0].name
        #print(hdf_mod_path.exists())
        with h5py.File(hdf_mod_path, 'r+') as hdf_mod:
            hdf_mod[f'{hdf_file}/plotselect/{plot}'][...] = dset_mean
        
    #print(dset_mean.shape) 
    hdf_base.close()
    
print('Summary hdfs output to: ', out_sum_dir)
    

Summary hdfs output to:  C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs\summary_hdfs


In [8]:
"""
#TEMP CELL, CAN BE DELETED FOR FINAL RELEASE, JUST IF NEED TO REFIT CERTAIN FILES

# Refits of silicon drift detector cell pellets
hdf_sublist = ['001_002_stitch', '001_004_stitch',  'sample001_0001', 'sample001_0003']

for hdf_file in hdf_sublist:
    print(hdf_file)
    xrf_deconvolute(hdf_file, 'falconx_det0')
    xrf_deconvolute(hdf_file, 'fluodet_det0')
    
print('deconvolutions complete')
"""

"\n#TEMP CELL, CAN BE DELETED FOR FINAL RELEASE, JUST IF NEED TO REFIT CERTAIN FILES\n\n# Refits of silicon drift detector cell pellets\nhdf_sublist = ['001_002_stitch', '001_004_stitch',  'sample001_0001', 'sample001_0003']\n\nfor hdf_file in hdf_sublist:\n    print(hdf_file)\n    xrf_deconvolute(hdf_file, 'falconx_det0')\n    xrf_deconvolute(hdf_file, 'fluodet_det0')\n    \nprint('deconvolutions complete')\n"

In [9]:
"""
#TEMP CELL, CAN BE DELETED FOR FINAL RELEASE, JUST IF NEED TO REFIT CERTAIN FILES

# Refits of GeCMOS detector breast cancers
hdf_sublist = ['sample304_0001', 'sample304_0002', 'sample304_b_0001', 'sample304_b_0002', 'sample304_b_0003', 'sample304_b_0004']

for hdf_file in hdf_sublist:
    print(hdf_file)
    xrf_deconvolute(hdf_file, 'falconx_det0', refit_state = True, weight=1)
    
print('deconvolutions complete')
"""

"\n#TEMP CELL, CAN BE DELETED FOR FINAL RELEASE, JUST IF NEED TO REFIT CERTAIN FILES\n\n# Refits of GeCMOS detector breast cancers\nhdf_sublist = ['sample304_0001', 'sample304_0002', 'sample304_b_0001', 'sample304_b_0002', 'sample304_b_0003', 'sample304_b_0004']\n\nfor hdf_file in hdf_sublist:\n    print(hdf_file)\n    xrf_deconvolute(hdf_file, 'falconx_det0', refit_state = True, weight=1)\n    \nprint('deconvolutions complete')\n"